#### 1.Imports


In [35]:
import requests
import json
from bs4 import BeautifulSoup

#### 2. Sending HTTP request to access first page with options

#### 3.Checking the code of HTTP response

In [ ]:
response.status_code

#### 4.Parse the HTML code of first page with options

In [ ]:
page_don = BeautifulSoup(response.text, 'html.parser')
print(page_don)

#### 5.Extract required data from parsed code

In [ ]:
opinions = page_don.select("div.js_product-review:not(.user-post--highlight)")
opinion = page_don.select_one("div.js_product-review:not(.user-post--highlight)")
print(type(page_don))
print(type(opinions))
print(len(opinions))
print(type(opinion))
print(opinion.prettify())

In [ ]:
opinions = page_don.select("div.js_product-review:not(.user-post--highlight)")
for opinion in opinions:
    opinion_id = opinion["data-entry-id"]
author = opinion.select_one("span.user-post__author-name").get_text()
try:
    recommendation = opinion.select_one("span.user-post__author-recommendation > em").get_text()
except AttributeError:
    recommendation = None
stars = opinion.select_one("span.user-post__score-count").get_text()
content = opinion.select_one("div.user-post__text").get_text()
try:
    pros = [p.get_text() for p in opinion.select_one("div.review-feature__item--positive")]
except TypeError: None
try:
    cons = [c.get_text() for c in opinion.select_one("div.review-feature__item--negative")]
except TypeError: None
vote_yes = opinion.select_one("button.vote-yes")["data-total-vote"]
vote_no = opinion.select_one("button.vote-no")["data-total-vote"]
published = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
try:
    purchased = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
except TypeError: None
print(opinion_id, author,recommendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep='\n')

#### 6. If there are more pages, move to the next page and repeat step 2-5 for it

In [ ]:
try:
    next_page ='https://www.ceneo.pl' + page_don.select_one('a.pagination__next')['href']
except TypeError:
    next_page = None


In [ ]:
while next_page:
    response = requests.get(next_page)
    if response.status_code == 200:
        page_don = BeautifulSoup(response.text, 'html.parser')
        opinions = page_don.select("div.js_product-review:not(.user-post--highlight)")
    for opinion in opinions:
        single_opinion = {}
        opinion_id = opinion["data-entry-id"]
    author = opinion.select_one("span.user-post__author-name").get_text()
    try:
        recommendation = opinion.select_one("span.user-post__author-recommendation > em").get_text()
    except AttributeError:
        recommendation = None
    stars = opinion.select_one("span.user-post__score-count").get_text()
    content = opinion.select_one("div.user-post__text").get_text()
    try:
        pros = [p.get_text() for p in opinion.select_one("div.review-feature__item--positive")]
    except TypeError: None
    try:
        cons = [c.get_text() for c in opinion.select_one("div.review-feature__item--negative")]
    except TypeError: None
    vote_yes = opinion.select_one("button.vote-yes")["data-total-vote"]
    vote_no = opinion.select_one("button.vote-no")["data-total-vote"]
    published = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"]
    try:
        purchased = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"]
    except TypeError: None
    print(opinion_id, author,recommendation, stars, content, pros, cons, vote_yes, vote_no, published, purchased, sep='\n')
    try:
        next_page ='https://www.ceneo.pl' + page_don.select_one('a.pagination__next')['href']
    except TypeError:
        next_page = None


In [ ]:
print(response.text)